In [1]:
!pip install google-cloud-storage

path_to_secret_key='secrets/gcp-rehab-ai-secret.json'

In [14]:
BUCKET_NAME = "rehab-image-detection-data"
FIXER_FOLDER_NAME = "fixer-upper"
RENOVATED_FOLDER_NAME = "renovated"

In [8]:
from google.cloud import storage

def list_top_level_folders_in_gcs(bucket_name, path_to_secret_key):
    """
    Lists top-level "folders" in a GCS bucket.

    :param bucket_name: Name of the GCS bucket.
    :param path_to_secret_key: Path to the service account key.
    :return: List of top-level "folders" in the bucket.
    """
    
    # Initialize the GCS client
    storage_client = storage.Client.from_service_account_json(path_to_secret_key)
    bucket = storage_client.bucket(bucket_name)

    # Get all blobs
    blobs = bucket.list_blobs()
    
    # Extract top-level "folders" from blob names
    folders = set()
    for blob in blobs:
        parts = blob.name.split('/')
        if len(parts) > 1:
            folders.add(parts[0])

    return list(folders)

# Example usage
gcs_folders = list_top_level_folders_in_gcs(BUCKET_NAME, path_to_secret_key)
print(gcs_folders)


['dvc-store', 'fixer-upper-processed', 'unlabeled', 'renovated', 'fixer-upper', 'labeled', 'renovated-processed']


In [7]:
# storage_client = storage.Client.from_service_account_json(path_to_secret_key)
# bucket = storage_client.bucket('rehab-image-detection-data')
# for i in bucket.list_blobs(delimiter='/'):
#     print(i)





# # blobs = bucket.list_blobs()
# # for blob in blobs:
# #     print(blob.name)


In [15]:
from google.cloud import storage

def list_files_in_gcs_folder(bucket_name, folder_prefix):
    """
    Lists all file paths in a specific GCS folder.

    :param bucket_name: Name of the GCS bucket.
    :param folder_prefix: Folder name (prefix) in GCS.
    :return: List of file paths within the folder.
    """
    
    # Initialize the GCS client
    # storage_client = storage.Client.from_service_account_json('secrets/secret_key_fixer_upper.json')
    storage_client = storage.Client.from_service_account_json(path_to_secret_key)
    bucket = storage_client.bucket(bucket_name)
    
    # Fetch all blob names with the given folder prefix
    blob_names = [blob.name for blob in bucket.list_blobs(prefix=folder_prefix)]
    
    return blob_names

# Example usage
# file_paths = list_files_in_gcs_folder('fixer-upper', 'images')  # fixer-upper is the bucket name, images is the folder name. images has two other folders within it called renovated and fixer_uppers
file_paths = list_files_in_gcs_folder(BUCKET_NAME, FIXER_FOLDER_NAME)
print(file_paths)


['fixer-upper-processed/', 'fixer-upper/', 'fixer-upper/.DS_Store', 'fixer-upper/7648106209/image_1.jpg', 'fixer-upper/7650055748/image_1.jpg', 'fixer-upper/7650055748/image_2.jpg', 'fixer-upper/7650055748/image_3.jpg', 'fixer-upper/7650055748/image_4.jpg', 'fixer-upper/7650055748/image_5.jpg', 'fixer-upper/7650055748/image_6.jpg', 'fixer-upper/7650055748/image_7.jpg', 'fixer-upper/7650055748/image_8.jpg', 'fixer-upper/7650055748/image_9.jpg', 'fixer-upper/7652036027/image_1.jpg', 'fixer-upper/7652036027/image_10.jpg', 'fixer-upper/7652036027/image_11.jpg', 'fixer-upper/7652036027/image_12.jpg', 'fixer-upper/7652036027/image_13.jpg', 'fixer-upper/7652036027/image_14.jpg', 'fixer-upper/7652036027/image_15.jpg', 'fixer-upper/7652036027/image_16.jpg', 'fixer-upper/7652036027/image_17.jpg', 'fixer-upper/7652036027/image_18.jpg', 'fixer-upper/7652036027/image_19.jpg', 'fixer-upper/7652036027/image_2.jpg', 'fixer-upper/7652036027/image_3.jpg', 'fixer-upper/7652036027/image_4.jpg', 'fixer-upp

In [10]:
import os
from google.cloud import storage

def upload_folder_to_gcs(bucket_name, source_folder_path, destination_folder_path):
    """
    Uploads a local folder to Google Cloud Storage.

    :param bucket_name: Name of the GCS bucket.
    :param source_folder_path: Local path to the folder you want to upload.
    :param destination_folder_path: Path in GCS where the folder should be uploaded.
    """
    
    # Initialize the GCS client
    storage_client = storage.Client.from_service_account_json(path_to_secret_key)
    bucket = storage_client.bucket(bucket_name)
    
    # Check if the root directory already exists in GCS
    root_directory_blob = bucket.blob(destination_folder_path)
    
    # Walk through each file in the local folder
    for dirpath, dirnames, filenames in os.walk(source_folder_path):
        for filename in filenames:
            # Construct the full local path
            local_file = os.path.join(dirpath, filename)
            
            # Construct the full GCS path
            remote_path = os.path.join(destination_folder_path, os.path.relpath(local_file, source_folder_path))
            
            if remote_path not in file_paths:
            # Upload the file
                blob = bucket.blob(remote_path)
                blob.upload_from_filename(local_file)
                print(f"File {local_file} uploaded to {remote_path} in bucket {bucket_name}.")
            else:
                print('file exists')


In [16]:
file_paths = list_files_in_gcs_folder(BUCKET_NAME, FIXER_FOLDER_NAME) #bucket name and folder name
print(file_paths)
upload_folder_to_gcs(bucket_name=BUCKET_NAME, source_folder_path=FIXER_FOLDER_NAME, destination_folder_path=FIXER_FOLDER_NAME)


['fixer-upper-processed/', 'fixer-upper/', 'fixer-upper/.DS_Store', 'fixer-upper/7648106209/image_1.jpg', 'fixer-upper/7650055748/image_1.jpg', 'fixer-upper/7650055748/image_2.jpg', 'fixer-upper/7650055748/image_3.jpg', 'fixer-upper/7650055748/image_4.jpg', 'fixer-upper/7650055748/image_5.jpg', 'fixer-upper/7650055748/image_6.jpg', 'fixer-upper/7650055748/image_7.jpg', 'fixer-upper/7650055748/image_8.jpg', 'fixer-upper/7650055748/image_9.jpg', 'fixer-upper/7652036027/image_1.jpg', 'fixer-upper/7652036027/image_10.jpg', 'fixer-upper/7652036027/image_11.jpg', 'fixer-upper/7652036027/image_12.jpg', 'fixer-upper/7652036027/image_13.jpg', 'fixer-upper/7652036027/image_14.jpg', 'fixer-upper/7652036027/image_15.jpg', 'fixer-upper/7652036027/image_16.jpg', 'fixer-upper/7652036027/image_17.jpg', 'fixer-upper/7652036027/image_18.jpg', 'fixer-upper/7652036027/image_19.jpg', 'fixer-upper/7652036027/image_2.jpg', 'fixer-upper/7652036027/image_3.jpg', 'fixer-upper/7652036027/image_4.jpg', 'fixer-upp

KeyboardInterrupt: 

In [7]:
file_paths = list_files_in_gcs_folder(BUCKET_NAME, RENOVATED_FOLDER_NAME) #bucket name and folder name
upload_folder_to_gcs(BUCKET_NAME , RENOVATED_FOLDER_NAME, RENOVATED_FOLDER_NAME)


File renovated/.DS_Store uploaded to renovated/.DS_Store in bucket rehab-image-detection-data.
File renovated/7659020525/image_15.jpg uploaded to renovated/7659020525/image_15.jpg in bucket rehab-image-detection-data.
File renovated/7659020525/image_14.jpg uploaded to renovated/7659020525/image_14.jpg in bucket rehab-image-detection-data.
File renovated/7659020525/image_16.jpg uploaded to renovated/7659020525/image_16.jpg in bucket rehab-image-detection-data.
File renovated/7659020525/image_17.jpg uploaded to renovated/7659020525/image_17.jpg in bucket rehab-image-detection-data.
File renovated/7659020525/image_13.jpg uploaded to renovated/7659020525/image_13.jpg in bucket rehab-image-detection-data.
File renovated/7659020525/image_12.jpg uploaded to renovated/7659020525/image_12.jpg in bucket rehab-image-detection-data.
File renovated/7659020525/image_10.jpg uploaded to renovated/7659020525/image_10.jpg in bucket rehab-image-detection-data.
File renovated/7659020525/image_11.jpg uploa

KeyboardInterrupt: 